In [3]:
# inspiration für scrollable presentation: 
# https://medium.com/@Ben_Obe/introduction-to-presenting-with-juypter-with-reveal-js-8e34a07081b2

from notebook.services.config import ConfigManager
cm = ConfigManager()

cm.update('livereveal', {
              'width': 1000,
              'height': 600,
              'scroll': True,
})

{'width': 1000, 'height': 600, 'scroll': True}

In [4]:
def print_content(filename):
    f = open(filename, 'r')
    file_contents = f.read()
    print(file_contents)
    f.close()

# Dokumentation für meine Implementation des Papers von Sebastian

Hier möchte ich die importierten Funktionalitäten beschreiben, die sich primär aus dem Paper "Least squares approximate policy iteration for learning bid prices in choice-based revenue management" von Sebastian Koch ergeben.

Das Dokument dient rein der Dokumentation. Die jeweils aktuellste Implementierung findet sich in *func_Koch.py*.

## Vorbemerkungen

Es gibt in meinem Modell:
* i = 0, ..., m-1 Ressourcen
* j = 0, ..., n-1 Produkte
* l = 0, ..., L-1 Kundensegmente
* t = 0, ..., T-1 Zeiten

Ich verwende folgende Daten als **Inputvariablen**:

| Name | Datentyp | Beschreibung |
| :--- | --- | :--- |
| **resources** | np.array | Ressourcen von 0 bis m-1 |
| capacities | np.array | Kapazitäten für alle Ressourcen |
| **products** | np.array | Produkte von 0 bis n-1 |
| revenues | np.array | Revenues für alle Produkte |
| A | np.array | Kapazitätsbedarfsmatrix mit m Zeilen und n Spalten: a_{ij} = 1 wenn Ressource i für Produkt j gebraucht wird |
| **customer_segments** | np.array | Kundensegmente von 0 bis L-1 |
| preference_weights | np.array | Matrix mit L Zeilen: jede Zeile l enthält die Präferenzen von Kundensegment l für alle Produkte |
| preference_no_purchase | np.array | Nichtkauf-Präferenzen für alle Kundensegmente |
| arrival_probabilities | np.array | Vektor mit Ankunftswahrscheinlichkeiten für alle Kundensegmente |
| **times** | np.array | Zeiten von 0 bis T-1 |

## Beispieldaten

Um die Korrektheit der nachstehenden Methoden überprüfen zu können, möchte ich hier den Beispieldatensatz aus der Sektion 4.2. von Koch verwenden (single-leg flight example).

In [5]:
import numpy as np

example = "singleLegFlight"

if example == "singleLegFlight":
    n = 4
    products = np.arange(n)
    revenues = np.array([1000, 800, 600, 400])

    T = 400
    times = np.arange(T)

    L = 1
    customer_segments = np.arange(L)
    arrival_probabilities = np.array([0.5])
    preference_weights = np.array([[0.4, 0.8, 1.2, 1.6]])

    varNoPurchasePreferences = np.array([1, 2, 3])
    preference_no_purchase = np.array([varNoPurchasePreferences[0]])

    m = 1
    resources = np.arange(m)

    varCapacity = np.arange(40, 120, 20)
    capacities = np.array([varCapacity[0]])

    # capacity demand matrix A (rows: resources, cols: products)
    # a_ij = 1 if resource i is used by product j
    A = np.array([[1, 1, 1, 1]])

# %% Check up
print("Check of dimensions: \n  -------------------")
print("Ressourcen: \t", len(resources) == len(capacities) == A.shape[0])
print("Produkte: \t", len(products) == len(revenues) == preference_weights.shape[1] == A.shape[1])
print("Kundensgemente:\t", len(customer_segments) == len(arrival_probabilities) == preference_weights.shape[0] ==
      len(preference_no_purchase))

def get_data_for_table1():
    return resources, \
           products, revenues, A, \
           customer_segments, preference_weights, arrival_probabilities, \
           times

Check of dimensions: 
  -------------------
Ressourcen: 	 True
Produkte: 	 True
Kundensgemente:	 True


## Import der nötigen Bibliotheken

In [6]:
#  PACKAGES
# Data
import numpy as np
import pandas as pd

# Calculation and Counting
import itertools
import math

# Memoization
import functools


In [7]:
from func_Koch import *
import inspect

Check of dimensions: 
  ------------------------
Ressourcen: 	 True
Produkte: 		 True
Kundensgemente:	 True
      rem cap        offer set
0   [2, 2, 1]     [1, 3, 4, 5]
1   [2, 1, 2]  [1, 2, 3, 4, 6]
2   [1, 2, 2]     [1, 3, 4, 5]
3   [2, 1, 1]        [1, 3, 4]
4   [1, 2, 1]     [1, 3, 4, 5]
5   [1, 1, 2]  [1, 2, 3, 4, 6]
6   [2, 1, 0]           [1, 3]
7   [2, 0, 1]        [2, 3, 4]
8   [1, 2, 0]        [1, 3, 5]
9   [0, 2, 1]        [1, 4, 5]
10  [1, 0, 2]     [2, 3, 4, 6]
11  [0, 1, 2]           [1, 4]
12  [1, 1, 1]        [1, 3, 4]
13  [1, 1, 0]           [1, 3]
14  [1, 0, 1]        [2, 3, 4]
15  [0, 1, 1]           [1, 4]
16  [1, 0, 0]              [3]
17  [0, 1, 0]              [1]
18  [0, 0, 1]              [4]
      rem cap        offer set
0   [3, 2, 2]     [1, 3, 4, 5]
1   [2, 3, 2]     [1, 3, 4, 5]
2   [2, 2, 3]  [1, 2, 3, 4, 5]
3   [3, 2, 1]     [1, 3, 4, 5]
4   [3, 1, 2]  [1, 2, 3, 4, 6]
5   [2, 3, 1]     [1, 3, 4, 5]
6   [1, 3, 2]     [1, 3, 4, 5]
7   [2, 1, 3]  [1, 2, 3,

## Hilfsmethoden

Zunächst definiere ich einige Hilfsmethoden.

* memoize: Dient der späteren Memoisierung von Methoden (was zu beeindruckender Performanzverbesserung führt).
* offer_sets(products): Generates all possible offer sets.

In [8]:
lines = inspect.getsource(memoize)
print(lines)

def memoize(func):
    cache = func.cache = {}

    @functools.wraps(func)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return memoizer



In [9]:
# %% HELPER-FUNCTIONS
def memoize(func):
    cache = func.cache = {}

    @functools.wraps(func)

    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return memoizer

def offer_sets(products):
    """
    Generates all possible offer sets.

    :param products:
    :return:
    """
    n = len(products)
    offer_sets_all = np.array(list(map(list, itertools.product([0, 1], repeat=n))))
    offer_sets_all = offer_sets_all[1:]  # always at least one product to offer
    return offer_sets_all

## Funktionen

Hier folgen die eigentlichen Funktionen.
* customer_choice_individual(offer_set_tuple, preference_weights, preference_no_purchase): 
For one customer of one customer segment, determine its purchase probabilities given one offer set.
* customer_choice_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
From perspective of retailer: With which probability can he expect to sell each product (respectively non-purchase)

Die folgenden Funktionen sind in einer Art und Weise implementiert, sodass sie bspw. Table 1 nachbauen können und dennoch wenige Parameter in der memoize-Methode abgespeichert werden müssen (also Kapazitäten, Präferenzen für Nicht-Kauf und Zeit t als Parameter).
* def delta_value_j(j, capacities, t, preference_no_purchase):
For one product j, what is the difference in the value function if we sell one product.
* def value_expected(capacities, t, preference_no_purchase):
Recursive implementation of the value function, i.e. dynamic program (DP) as described on p. 241.

### customer_choice_individual

Gegeben sei ein Offerset: *x* $\in \{0,1\}^n$. 

Ein Kunde aus *Kundensegment l* mit Präferenz von $u_{lj}$ für *Produkt j* und Nichtkaufpräferenz von $u_{l0}$ erwirbt  Produkt j mit Wkeit

$p_{lj}(x) = \frac{u_{lj}x_j}{u_{l0} + \sum_{p\in[n]} u_{lp}x_p}$ .

Die Nichtkaufwahrscheinlichkeit beträgt dementsprechend

$p_{l0}(x) = 1 - \sum_{p\in[n]} p_{lj}(x)$ .

### customer_choice_vector

Der Retailer legt lediglich das Offerset *x* fest und kann dann die folgende Käufe erwarten keinenicht Wkeiten, da Summe nicht 1 ergeben muss).

$p_j(x) = \sum_{l\in[L]}\lambda_l p_{lj}(x)$ .

Die Nichtkaufwahrscheinlichkeit beträgt entsprechend

$p_0(x) = 1 - \sum_{p\in[n]} p_j(x)$ .

In [10]:
# %% FUNCTIONS
@memoize
def customer_choice_individual(offer_set_tuple, preference_weights, preference_no_purchase):
    """
    For one customer of one customer segment, determine its purchase probabilities given one offer set.

    Tuple needed for memoization.

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered
    :param preference_weights: preference weights of one customer
    :param preference_no_purchase: no purchase preference of one customer
    :return: array of purchase probabilities ending with no purchase
    """

    if offer_set_tuple is None:
        ret = np.zeros_like(preference_weights)
        return np.insert(ret, -1, 1)

    offer_set = np.asarray(offer_set_tuple)
    ret = preference_weights * offer_set
    ret = np.array(ret / (preference_no_purchase + sum(ret)))
    ret = np.insert(ret, -1, 1 - sum(ret))
    return ret

In [11]:
@memoize
def customer_choice_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    From perspective of retailer: With which probability can he expect to sell each product (respectively non-purchase)

    :param offer_set_tuple: tuple with offered products indicated by 1=product offered
    :param preference_weights: preference weights of all customers
    :param preference_no_purchase: preference for no purchase for all customers
    :param arrival_probabilities: vector with arrival probabilities of all customer segments
    :return: array with probabilities of purchase ending with no purchase
    TODO probabilities don't have to sum up to one?
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    for l in np.arange(len(preference_weights)):
        probs += arrival_probabilities[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                                       preference_no_purchase[l])
    return probs

### delta_value_j

### value_expected

Sei $x_t \in \{0,1\}^n$ ein mögliches Offerset zur Zeit $t$ bei gegebenen Kapazitäten $c \in \mathbb{N}_0^n$. Dann ergibt sich der erwartete Wert als 

$$V_t(c) = \max_{x_t \in \{0,1\}^n}\left\{ \sum_{j \in [n]} p_j(x_t) \left( r_j - \Delta_j V_{t+1}(c) \right) \right\} + V_{t+1}(c)\quad \forall t, c \geq 0$$

wobei $p_j(x_t)$ die insgesamte (über alle Kundensegmente) Kaufwkeit von Produkt $j$ darstellt und $\Delta_j V_{t+1}(c) := V_{t+1}(c) - V_{t+1}(c - a_j)$.

In [12]:
def delta_value_j(j, capacities, t, A, preference_no_purchase):
    """
    For one product j, what is the difference in the value function if we sell one product.

    :param j:
    :param capacities:
    :param t:
    :param preference_no_purchase:
    :return:
    """
    return value_expected(capacities, t, preference_no_purchase)[0] - \
        value_expected(capacities - A[:, j], t, preference_no_purchase)[0]

In [13]:
@memoize
def value_expected(capacities, t, preference_no_purchase):
    """
    Recursive implementation of the value function, i.e. dynamic program (DP) as described on p. 241.

    :param capacities:
    :param t: time to go (last chance for revenue is t=0)
    :param preference_no_purchase:
    :return: value to be expected and optimal policy
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_for_table1()
    T = len(times)

    offer_sets_to_test = offer_sets(products)

    max_index = 0
    max_val = 0

    if all(capacities == 0):
        return 0, None
    if any(capacities < 0):
        return -math.inf, None
    if t == T:
        return 0, None

    for offer_set_index in range(len(offer_sets_to_test)):
        offer_set = offer_sets_to_test[offer_set_index, :]
        probs = customer_choice_vector(tuple(offer_set), preference_weights, preference_no_purchase,
                                       arrival_probabilities)

        val = float(value_expected(capacities, t + 1, preference_no_purchase)[0])  # ohne "float" würde ein numpy array
        #  zurückgegeben, und das später (max_val = val) direkt auf val verknüpft (call by reference)
        for j in products:  # nett, da Nichtkaufalternative danach kommt und nicht betrachtet wird
            p = float(probs[j])
            if p > 0.0:
                value_delta_j = delta_value_j(j, capacities, t + 1, A, preference_no_purchase)
                val += p * (revenues[j] - value_delta_j)

        if val > max_val:
            max_index = offer_set_index
            max_val = val
    return max_val, tuple(offer_sets_to_test[max_index, :])

## Ein paar Ergebnisse

Die Ergebnisse sehen ganz brauchbar aus. Wenn sie auch nicht exakt mit denen der Table 1 übereinstimmen. Warum nicht? Beachte die Code-Feinheiten mit "float" in value_expected().

<span style="color:green"> Ergebnisse stimmen teilweise mit Table 1 in Sebastians Paper überein, teilweise nicht. Siehe auch Anmerkung im Paper. Nach Absprache mit Sebastian (11.4.), sind meine Ergebnisse korrekt.</span>

In [14]:
dftable1 = pd.read_pickle("table1_DP.pkl")
dftable1

,c,u,DP,CDLP
0,[40],[1],"(39995.18483814307, (1, 0, 0, 0))","({1: ((1, 0, 0, 0), 279.99999999999994)}, 4000..."
1,[40],[2],"(38013.67099976408, (1, 0, 0, 0))","({1: ((1, 0, 0, 0), 336.0), 2: ((1, 1, 0, 0), ..."
2,[40],[3],"(35952.031147207534, (1, 1, 0, 0))","({1: ((1, 0, 0, 0), 204.0), 3: ((1, 1, 0, 0), ..."
3,[60],[1],"(58454.753775844474, (1, 0, 0, 0))","({1: ((1, 0, 0, 0), 378.00000000000006), 2: ((..."
4,[60],[2],"(53227.23269230522, (1, 1, 0, 0))","({2: ((1, 1, 0, 0), 256.0), 3: ((1, 0, 0, 0), ..."
5,[60],[3],"(49977.054186201975, (1, 1, 0, 0))","({4: ((1, 1, 1, 0), 36.00000000000005), 5: ((1..."
6,[80],[1],"(73126.21245972312, (1, 1, 0, 0))","({1: ((1, 0, 0, 0), 224.00000000000003), 2: ((..."
7,[80],[2],"(66170.9528230564, (1, 1, 0, 0))","({2: ((1, 1, 0, 0), 341.3333333333333), 3: ((1..."
8,[80],[3],"(59969.77918696259, (1, 1, 1, 0))","({4: ((1, 1, 1, 0), 288.00000000000006), 5: ((..."
9,[100],[1],"(87233.33717875695, (1, 1, 0, 0))","({2: ((1, 0, 0, 0), 69.99999999999993), 3: ((1..."


# Parallel Flight Discussion

Aufbauend auf dem Ansatz von Miranda und Bront habe ich deren *choice based deterministic linear program (CDLP)* (p. 775) implementiert und deren *dynamic programming decomposition (DPD)*. Hier beschreiben wir zunächst den Beispieldatensatz parallel flights, bevor wir die Implementation vorstellen und die Ergebnisse für Table 3 darlegen.

## Example Three Parallel Flights (PF)

In [15]:
n = 6
products = np.arange(n)
revenues = np.array([400, 800, 500, 1000, 300, 600])

T = 300
times = np.arange(T)

L = 4
customer_segments = np.arange(L)
arrival_probabilities = np.array([0.1, 0.15, 0.2, 0.05])
preference_weights = np.array([[0, 5, 0, 10, 0, 1],
                               [5, 0, 1, 0, 10, 0],
                               [10, 8, 6, 4, 3, 1],
                               [8, 10, 4, 6, 1, 3]])

var_no_purchase_preferences = np.array([[1, 5, 5, 1],
                                        [1, 10, 5, 1],
                                        [5, 20, 10, 5]])
preference_no_purchase = var_no_purchase_preferences[0]

m = 3
resources = np.arange(m)

base_capacity = np.array([30, 50, 40])
delta = np.arange(0.4, 1.21, 0.2)
var_capacities = np.zeros((len(delta), len(base_capacity)))
for i in np.arange(len(delta)):
    var_capacities[i] = delta[i]*base_capacity
capacities = var_capacities[0]

# capacity demand matrix A (rows: resources, cols: products)
# a_ij = 1 if resource i is used by product j
A = np.array([[1, 1, 0, 0, 0, 0],
              [0, 0, 1, 1, 0, 0],
              [0, 0, 0, 0, 1, 1]])

## Funktionen

Die folgenden Inhalte sind sehr stark an Bront et al orientiert, wobei ich einige <span style="color:red"> Details farblich markiere. </span>

### Hilfsfunktionen
Sei S ein Offerset, also $x \in \{0,1\}^n$ mit $x_j=1$ nur falls j im Offerset. 

$P_{lj}(S)$ sei die Wahrscheinlichkeit, dass ein Kunde aus Segment $l$ Produkt $j$ kauft, wenn Offerset $S$ angeboten wird. Nach dem *multinomial logit model (MNL)* gilt:
$$ P_{lj}(S) = \frac{u_{lj}x_j}{\sum_{p \in S} u_{lp} x_p + u_{l0}}~.$$

Hieraus folgt, dass <span style="color:red"> gegeben ein Kunde kommt </span> und Offerset S wird angeboten, die Verkaufswkeit für Produkt j sich errechnet als
$$ P_j(S) = \sum_{l \in [L]} p_l P_{lj}(S) ~,$$
woraus sich $P(S) = (P_1(S), ..., P_n(S))^T$ ergibt.

<span style="color:red"> So wie wir die arrival probabilities $\lambda_l$ abspeichern, müssen wir hier $p_l$ durch Normierung berechnen. </span>

Dann berechnet sich der erwartete Erlös für ein gegebenes Offerset als
$$ R(S) = \sum_{j \in S} r_j P_j(S)~.$$

Gegeben die Ankunft eines Kunden, dann sei $Q_i(S)$ die bedingte Wkeit, dass eine Einheit der Ressource $i$ genutzt wird, falls $S$ angeboten wird. Sie berechnet sich als
$$ Q(S) = A P(S)~.$$

In [16]:
def purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    P_j(S) for all j, P_0(S) at the end

    :param offer_set_tuple: S
    :param preference_weights
    :param preference_no_purchase
    :param arrival_probabilities
    :return: P_j(S) for all j, P_0(S) at the end
    """
    probs = np.zeros(len(offer_set_tuple) + 1)
    p = arrival_probabilities/(sum(arrival_probabilities))
    for l in np.arange(len(preference_weights)):
        probs += p[l] * customer_choice_individual(offer_set_tuple, preference_weights[l, :],
                                                   preference_no_purchase[l])
    return probs

In [17]:
def revenue(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities):
    """
    R(S)

    :param offer_set_tuple: S
    :return: R(S)
    """
    return sum(revenues * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                               arrival_probabilities)[:-1])

In [18]:
def quantity_i(offer_set_tuple, preference_weights, preference_no_purchase, arrival_probabilities, i):
    """
    Q_i(S)

    :param offer_set_tuple: S
    :param i: resource i
    :return: Q_i(S)
    """
    return sum(A[i, :] * purchase_rate_vector(offer_set_tuple, preference_weights, preference_no_purchase,
                                              arrival_probabilities)[:-1])

### CDLP-base version

Die folgende Implementierung nimmt die gesamte Probleminstanz als Parameter und zusätzlich die zu betrachtenden Offersets $N$. Es soll folgendes Optimierungsproblem gelöst werden.

\begin{align}
V^{CDLP}(N) = \max & \sum_{S\in N} \lambda R(S) t(S) \\
s.t. & \sum_{S\in N} \lambda Q(S) t(S) \leq c~, \\
& \sum_{S\in N} t(S) \leq T~,\\
& t(S) \geq 0 \quad \forall S \in N~.
\end{align}

<span style="color:red"> Beachte, dass $\lambda = \sum_{l \in [L]} \lambda_l$ die Ankunftsrate irgendeines Kunden wiedergibt. </span>

<span style="color:green"> Mit dem Datensatz aus _example0_ läuft die Funktion korrekt durch und spuckt dieselben Ergebnisse wie in Bront et al. p. 774 links oben aus. </span>

In [19]:
def CDLP(offer_sets: np.ndarray):
    """
    Implements (4) of Bront et al. Needs the offer-sets to look at (N) as input.

    :param offer_sets: N
    :return: dictionary of (offer set, time offered),
    """
    resources, capacities, \
        products, revenues, A, \
        customer_segments, preference_weights, preference_no_purchase, arrival_probabilities, \
        times = get_data()

    offer_sets = pd.DataFrame(offer_sets)

    S = {}
    R = {}
    Q = {}
    for index, offer_array in offer_sets.iterrows():
        S[index] = tuple(offer_array)
        R[index] = revenue(tuple(offer_array), preference_weights, preference_no_purchase, arrival_probabilities)
        temp = {}
        for i in resources:
            temp[i] = quantity_i(tuple(offer_array), preference_weights, preference_no_purchase,
                                 arrival_probabilities, i)
        Q[index] = temp

    try:
        m = Model()

        # Variables
        mt = m.addVars(offer_sets.index.values, name="t", lb=0.0)  # last constraint

        # Objective Function
        m.setObjective(lam * quicksum(R[s] * mt[s] for s in offer_sets.index.values), GRB.MAXIMIZE)

        mc = {}
        # Constraints
        for i in resources:
            mc[i] = m.addConstr(lam * quicksum(Q[s][i] * mt[s] for s in offer_sets.index.values), GRB.LESS_EQUAL,
                                capacities[i],
                                name="constraintOnResource")
        msigma = m.addConstr(quicksum(mt[s] for s in offer_sets.index.values), GRB.LESS_EQUAL, T)

        m.optimize()

        ret = {}
        pat = r".*?\[(.*)\].*"
        for v in m.getVars():
            if v.X > 0:
                match = re.search(pat, v.VarName)
                erg_index = match.group(1)
                ret[int(erg_index)] = (tuple(offer_sets.loc[int(erg_index)]), v.X)
                print(offer_sets.loc[int(erg_index)], ": ", v.X)

        dualPi = np.zeros_like(resources)
        for i in resources:
            dualPi[i] = mc[i].pi
        dualSigma = msigma.pi

        valOpt = m.objVal

        return ret, valOpt, dualPi, dualSigma

    except GurobiError:
        print('Error reported')

In [20]:
print_content('res_CDLP.txt')

Academic license - for non-commercial use only
Optimize a model with 4 rows, 255 columns and 927 nonzeros
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [6e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+01]
Presolve time: 0.00s
Presolved: 4 rows, 255 columns, 927 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.1657662e+34   5.943074e+32   9.165766e+04      0s
      10    1.1409091e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds
Optimal objective  1.140909091e+04
0    0
1    1
2    1
3    0
4    0
5    0
6    0
7    0
Name: 95, dtype: int32 :  1.9999999999999947
0    1
1    1
2    1
3    0
4    0
5    0
6    0
7    0
Name: 223, dtype: int32 :  16.992628992628998
0    1
1    1
2    1
3    1
4    0
5    1
6    0
7    0
Name: 243, dtype: int32 :  11.007371007371008



### CDLP by column generation

Die obere Methode (über alle möglichen Offersets) leidet darunter, dass die Anzahl an Offersets exponentiell mit der Anzahl an Produkten $n$ wächst ($2^n-1$). Ein schlauer Ansatz ist die Möglichkeit der Column Generation, bei welcher nach und nach die vielversprechendsten Offersets als mögliche Variablen zum CDLP-Optimierungsproblem hinzugefügt werden. Um dies anzugehen, möchten wir zwei Ansätze vorstellen. Einen exakten Ansatz (MIP Formulation) und eine Heuristik (Greedy Heuristic).

Bei beiden Hilffunktionen ist die anwesende Kapazität völlig egal. Lediglich Preise (revenues und duale Preise der Ressourcen) und Präferenzen (für Produkte und Nicht-Kauf) sind ausschlaggebend.

<span style="color:green"> Sowohl für MIP Formulierung als auch für Greedy Heuristik werden die Ergebnisse aus Bront p. 775 Example of nonoptimality of the greedy heuristik exakt repliziert. </span>

#### Hilfsfunktionen

##### MIP Formulation

Mit K groß lösen wir:

\begin{align}
\max ~ & \sum_{l \in [L]}\sum_{j \in [n]} \lambda_l (r_j - A_j^T \pi) u_{lj} z_{lj} \\
s.t.~~ & x_l v_{l0} + \sum_{j \in [n]} u_{lj} z_{lj} = 1 \quad \forall l \in [L]~, \\
& x_l - z_{lj} \leq K - K y_j \quad \forall l \in [L], j \in [n]~, \\
& z_{lj} \leq x_l  \quad \forall l \in [L], j \in [n]~, \\
& z_{lj} \leq K y_j  \quad \forall l \in [L], j \in [n]~, \\
& y_j \in \{0,1\} \quad x_l \geq 0 \quad z_{lj} \geq 0~.
\end{align}

Beachte, dass $u_{lj}=0$, falls Kunde aus Segment $l$ Produkt $j$ nicht in seinem Consideration Set hat.


In [21]:

def column_MIP(pi, w=0):  # pass w to test example for greedy heuristic
    """
    Implements MIP formulation on p. 775 lhs

    :param pi:
    :param w:
    :return: optimal tuple of products to offer
    """
    resources, capacities, \
        products, revenues, A, \
        customer_segments, preference_weights, preference_no_purchase, arrival_probabilities, \
        times = get_data()

    K = 1/min(preference_no_purchase.min(), np.min(preference_weights[np.nonzero(preference_weights)]))+1

    if isinstance(w, int) and w == 0:  # 'and' is lazy version of &
        w = np.zeros_like(revenues, dtype=float)
        for j in products:
            w[j] = revenues[j] - sum(A[:, j]*pi)

    try:
        m = Model()

        mx = {}
        my = {}
        mz = {}

        # Variables
        for j in products:
            my[j] = m.addVar(0, 1, vtype=GRB.BINARY, name="y["+str(j)+"]")
        for l in customer_segments:
            mx[l] = m.addVar(0.0, name="x["+str(l)+"]")
            temp = {}
            for j in products:
                temp[j] = m.addVar(0.0, name="z["+str(l)+","+str(j)+"]")
            mz[l] = temp

        # Objective
        m.setObjective(quicksum(arrival_probabilities[l] * w[j] * preference_weights[l, j] * mz[l][j]
                                for l in customer_segments for j in products), GRB.MAXIMIZE)

        # Constraints
        mc1 = m.addConstrs((mx[l]*preference_no_purchase[l] +
                            quicksum(preference_weights[l, j]*mz[l][j] for j in products) == 1
                            for l in customer_segments), name="mc1")
        mc2 = m.addConstrs((mx[l] - mz[l][j] <= K - K*my[j] for l in customer_segments for j in products),
                           name="mc2")
        mc3 = m.addConstrs((mz[l][j] <= mx[l] for l in customer_segments for j in products), name="mc3")
        mc4 = m.addConstrs((mz[l][j] <= K*my[j] for l in customer_segments for j in products), name="mc4")

        m.optimize()

        y = np.zeros_like(revenues)
        for j in products:
            y[j] = my[j].x

        return tuple(y)

    except GurobiError:
        print('Error reported')

In [22]:
print_content('res_MIP.txt')

Optimize a model with 30 rows, 15 columns and 71 nonzeros
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 8 rows and 4 columns
Presolve time: 0.00s
Presolved: 22 rows, 11 columns, 57 nonzeros
Variable types: 4 continuous, 7 integer (3 binary)

Root relaxation: objective 5.950000e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   59.50000    0    7          -   59.50000      -     -    0s
H    0     0                      53.5000000   59.50000  11.2%     -    0s
     0     0     cutoff    0        53.50000   53.50000  0.00%     -    0s

Cutting planes:
  Gomory: 2

Explored 1 nodes (10 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processo

#### Hilfsfunktionen

##### Greedy Heuristic

Ziel: Berechne Offerset $S$ mittels Heuristik.

Berechne $w_j = r_j - A^T\pi$

1. Für alle Produkte $j$ mit $w_j \leq 0$  setzen wir $y_j = 0$.
2. Definiere $S' \subset N$ mit allen Produkten $j$ sodass $w_j > 0$.
3. Füge das Produkt $j^*$ mit höchstem erwarteten Deckungsbeitrag zu $S$ hinzu.
$$ j^* = \arg \max_{j \in S'} \sum_{l \in [L]} \lambda_l \frac{w_j u_{lj}}{u_{lj} + u_{l0}} $$
Setze $S := \{j^*\}$, $S' := S' \backslash \{ j^*\}$.
4. Solange $S$ noch verändert wird, wiederhole
    1. Berechne 
    $$ j^* = \arg \max_{j \in S'} \sum_{l \in [L]} \lambda_l  \frac{\sum_{p \in S \cup \{j\}}w_p u_{lp}}{\sum_{p \in S \cup \{j\}}u_{lp} + u_{l0}} $$
    2. Falls $Wert(S \cup \{j^*\}) > Wert(S)$, dann setze $S := S \cup \{j^*\}$, $S' := S' \backslash \{ j^*\}$.
5. Gebe S zurück (also nur die aufgenommenen Produkte).

<span style="color:red"> $Wert(S \cup \{j^*\}) := \max_{j \in S'} \sum_{l \in [L]} \lambda_l  \frac{\sum_{p \in S \cup \{j\}}w_p u_{lp}}{\sum_{p \in S \cup \{j\}}u_{lp} + u_{l0}}$ ?? </span>

In [23]:
def column_greedy(pi, w=0):  # pass w to test example for greedy heuristic
    """
    Implements Greedy Heuristic on p. 775 rhs

    :param pi:
    :param w:
    :return: heuristically optimal tuple of products to offer
    """
    # Step 1
    y = np.zeros_like(revenues)

    if isinstance(w, int) and w == 0:  # and is lazy version of &
        w = np.zeros_like(revenues, dtype=float)  # calculate the opportunity costs
        for j in products:
            w[j] = revenues[j] - sum(A[:, j]*pi)

    # Step 2
    Sprime = set(np.where(w > 0)[0])

    # Step 3
    value_marginal = np.zeros_like(w, dtype=float)
    for j in Sprime:
        for l in customer_segments:
            value_marginal[j] += preference_weights[l, j]/(preference_weights[l, j] + preference_no_purchase[l])
        value_marginal[j] *= w[j]

    jstar = np.argmax(value_marginal)
    v_new = value_marginal[jstar]

    S = {jstar}
    Sprime = Sprime-S

    # Step 4
    while True:
        v_akt = copy.deepcopy(v_new)  # deepcopy to be on the safe side
        v_temp = np.zeros_like(revenues, dtype=float)  # uses more space then necessary, but simplifies indices below
        for j in Sprime:
            for l in customer_segments:
                z = 0
                n = 0
                for p in S.union({j}):
                    z += w[p]*preference_weights[l, p]
                    n += preference_weights[l, p]
                n += preference_no_purchase[l]
                v_temp[j] += arrival_probability[l]*z/n
        jstar = np.argmax(value_marginal)  # argmax always returns index of first maxima (if there is > 1)
        v_new = value_marginal[jstar]
        if v_new > v_akt:
            S = S.union({jstar})
            Sprime = Sprime - {jstar}
        else:
            break

    # Step 5
    y[list(S)] = 1
    return tuple(y), v_new

In [24]:
print_content('res_GreedyHeuristic.txt')

((1, 0, 0), 50.0)



#### CDLP by column generation

Wir folgen Bront et al. Zunächst wird mittels der greedy heuristic versucht, ein weiteres mögliches Offerset zu finden. Falls dies nicht möglich ist, wird das die MIP Formulierung gelöst. Findet diese auch kein neues spannendes Offerset, so haben wir das Optimum gefunden. Die dualen Variablen $pi, sigma$ sind initial $0$ und ergeben sich im weiteren aus der Berechnung des CDLP mit der jeweils letzten Menge $N$ an interessanten Offersets $S$.

<span style="color:green"> Die Ergebnisse aus Koch Tabelle 3 werden exakt repliziert. </span>

In [25]:
# CDLP by column generation
def CDLP_by_column_generation(capacities, preference_no_purchase):
    """
    Implements Bront et als approach for CDLP by column generation as pointed out on p. 775 just above "5. Decomp..."

    :return:
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    dual_pi = np.zeros(len(A))

    col_offerset, col_val = column_greedy(preference_no_purchase, dual_pi)
    if all(col_offerset == np.zeros_like(col_offerset)):
        print("MIP solution used to solve CDLP by column generation")
        col_offerset, col_val = column_MIP(preference_no_purchase, dual_pi)

    offer_sets = pd.DataFrame([np.array(col_offerset)])

    val_akt_CDLP = 0
    ret, val_new_CDLP, dual_pi, dual_sigma = CDLP(capacities, preference_no_purchase, offer_sets)

    while val_new_CDLP > val_akt_CDLP:
        val_akt_CDLP = copy.deepcopy(val_new_CDLP)  # deepcopy and new name to be on the safe side

        col_offerset, col_val = column_greedy(preference_no_purchase, dual_pi)
        if not offer_sets[(offer_sets == np.array(col_offerset)).all(axis=1)].index.empty:
            col_offerset, col_val = column_MIP(preference_no_purchase, dual_pi)
            if not offer_sets[(offer_sets == np.array(col_offerset)).all(axis=1)].index.empty:
                break  # nothing changed

        offer_sets = offer_sets.append([np.array(col_offerset)], ignore_index=True)
        ret, val_new_CDLP, dual_pi, dual_sigma = CDLP(capacities, preference_no_purchase, offer_sets)

    return ret, val_new_CDLP, dual_pi, dual_sigma

In [26]:
dftableCDLP = pd.read_pickle("table3_CDLP.pkl")
dftableCDLP

,c,u,CDLP
0,"[12.0, 20.0, 16.0]","[1, 5, 5, 1]",39200
1,"[12.0, 20.0, 16.0]","[1, 10, 5, 1]",39200
2,"[12.0, 20.0, 16.0]","[5, 20, 10, 5]",37571.1
3,"[18.0, 30.0, 24.0]","[1, 5, 5, 1]",56884.1
4,"[18.0, 30.0, 24.0]","[1, 10, 5, 1]",56848
5,"[18.0, 30.0, 24.0]","[5, 20, 10, 5]",53819.9
6,"[24.0, 40.0, 32.0]","[1, 5, 5, 1]",71936.4
7,"[24.0, 40.0, 32.0]","[1, 10, 5, 1]",71794.8
8,"[24.0, 40.0, 32.0]","[5, 20, 10, 5]",61868.2
9,"[30.0, 50.0, 40.0]","[1, 5, 5, 1]",79155.7


## Decomposition Approximation Algorithm

<span style="color:red"> Hierüber sprechen. Da in Bront nicht ganz klar, wie zB duale Preise für single leg DP gehandhabt werden. Talluri hilft hier nur bedingt, da keine dualen Preise betrachtet werden. </span>

Gehen wir weiter und implementieren die value of capacity. Dabei werden zunächst die Werte für die einzelnen Ressourcen approximiert und dann addiert.

### Herleitung

Seien $\pi = (\pi_1, ..., \pi_m)$ die Opportunitätskosten der $m$ Ressourcen. Dann ergibt sich als Approximation der Wertfunktion bei Kapazität $c \in \mathbb{N}_0^m$ und Zeit $t$
$$ V_t(c) \approx \hat{V}_t^i(c_i) + \sum_{k \neq i} \pi_k c_k$$
Als Opportunitätskosten für Produkt $j$ ergeben sich nun
\begin{align}
V_t(c) - V_t(c - A_j) & \approx \hat{V}_t^i(c_i) - \hat{V}_t^i(c_i - 1) + \sum_{k \in A_j, k \neq i} \pi_k~, \quad &\text{if } i \in A_j~, \\
V_t(c) - V_t(c - A_j) & \approx \sum_{k \in A_j} \pi_k~, \quad &\text{if } i \notin A_j~, \\
\end{align}
Mit $\Delta \hat{V}_t^i(c_i) := \hat{V}_t^i(c_i) - \hat{V}_t^i(c_i - 1)$ kann dies geschrieben werden als
$$ V_t(c) - V_t(c - A_j) \approx \left(\Delta \hat{V}_t^i(c_i) - \pi_i\right)\mathbb{1}_{i \in A_j} + \sum_{k \in A_j} \pi_k$$
Das klassische duale Programm kann approximiert werden. Vergleiche oben *value_expected*
\begin{align}
V_t(c) &= \max_{x_t \in \{0,1\}^n}\left\{ \sum_{j \in [n]} p_j(x_t) \left( r_j - \Delta_j V_{t+1}(c) \right) \right\} + V_{t+1}(c)\quad \forall t, c \geq 0 \\
V_t^i(c_i) &= \max_{x_t \in \{0,1\}^n}\left\{ \sum_{j \in [n]} \lambda P_j(x_t) \left( r_j - ( \Delta V_{t+1}^i(c_i) - \pi_i ) \mathbb{1}_{i \in A_j} - \sum_{k \in A_j} \pi_k \right) \right\} + V_{t+1}^i(c_i)\quad \forall t, c \geq 0 
\end{align}

### value_leg_i

Sei Offerset $S$ angeboten. Angenommen, es kommt ein Kunde, so wird $Q(S) = \sum_{j \in S} P_j(S)$ verkauft. <span style="color:red"> Beachte, hier nur Anzahl an verkauften Einheiten von Ressource i relevant. Also $Q^i(S) = \sum_{j \in S} a_{ij}P_j(S)$. </span> Für den Gewinn (Erlös abzüglich Kosten) ergibt sich ... 

Einsicht: $\lambda$ wurde bei Talluri eingeführt und von Bront übernommen. Macht es aber nicht gerade schön. => implementiere value_leg_i_11 direkt (nach Formel oben)

Ich implementiere die Formel (11) aus Bront et al, welche genau obiger Formel entspricht, hier noch einmal:
$$V_t^i(c_i) = \max_{x_t \in \{0,1\}^n}\left\{ \sum_{j \in [n]} \lambda P_j(x_t) \left( r_j - ( \Delta V_{t+1}^i(c_i) - \pi_i ) \mathbb{1}_{i \in A_j} - \sum_{k \in A_j} \pi_k \right) \right\} + V_{t+1}^i(c_i)\quad \forall t, c \geq 0 
$$

In [34]:
# leg level decomposition directly via (11)
@memoize
def value_leg_i_11(i, x_i, t, pi, preference_no_purchase):
    """
    Implements the table of value leg decomposition on p. 776

    :param i:
    :param x_i:
    :param t:
    :param pi:
    :return: optimal value, index of optimal offer set, tuple optimal offer set
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()
    T = len(times)
    lam = sum(arrival_probabilities)

    if t == T+1:
        return 0, None, None, None
    elif x_i <= 0:
        return 0, None, None, None

    offer_sets_all = get_offer_sets_all(products)
    offer_sets_all = pd.DataFrame(offer_sets_all)

    val_akt = 0.0
    index_max = 0

    df2 = pd.DataFrame({"purchase_rates": [[0]] * offer_sets_all.__len__()})
    df3 = pd.DataFrame({"temps": [[0]] * offer_sets_all.__len__()})

    for index, offer_array in offer_sets_all.iterrows():
        temp = np.zeros_like(products, dtype=float)
        for j in products:
            if offer_array[j] > 0:
                temp[j] = (revenues[j] -
                           (value_leg_i_11(i, x_i, t+1, pi, preference_no_purchase)[0] -
                            value_leg_i_11(i, x_i-1, t+1, pi, preference_no_purchase)[0] - pi[i]) * A[i, j] -
                           sum(pi[A[:, j] == 1]))
        val_new = sum(purchase_rate_vector(tuple(offer_array), preference_weights,
                                           preference_no_purchase, arrival_probabilities)[:-1] * temp)
        if val_new > val_akt:
            index_max = copy.copy(index)
            val_akt = copy.deepcopy(val_new)

        df2.loc[index, "purchase_rates"] = [purchase_rate_vector(tuple(offer_array), preference_weights,
                                           preference_no_purchase, arrival_probabilities)]
        df3.loc[index, "temps"] = [temp]
    return lam * val_akt + value_leg_i_11(i, x_i, t+1, pi, preference_no_purchase)[0], \
        index_max, tuple(offer_sets_all.iloc[index_max]), df2.loc[index_max, "purchase_rates"], df3.loc[index_max, "temps"]


### displacement_costs_vector and calculate_offer_set

Um nun die optimalen Offersets dynamisch zu bestimmen, muss zunächst der displacement Vektor $\Delta V_t(x)$ berechnet werden, gemäß
$$\Delta V_t^i(x) = \beta \Delta V_t^i(x_i) + (1-\beta)\pi_i^* \quad \forall i$$

Hieraus ergibt sich das optimale Offerset gemäß
$$\arg\max_{S\subset [n]}\left\{ \sum_{j \in S, j \text{ available}} \lambda P_j(S) (r_j - \Delta V_{t+1}^T(x) A_j)\right\} $$

In [35]:
def displacement_costs_vector(capacities_remaining, preference_no_purchase, t, pi, beta=1):
    """
    Implements the displacement vector on p. 777

    :param capacities_remaining:
    :param t:
    :param pi:
    :param beta:
    :return:
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations()

    delta_v = 1.0*np.zeros_like(resources)
    for i in resources:
        delta_v[i] = beta * (value_leg_i_11(i, capacities_remaining[i], t, pi, preference_no_purchase)[0] -
                             value_leg_i_11(i, capacities_remaining[i] - 1, t, pi, preference_no_purchase)[0]) + \
                     (1-beta) * pi[i]
    return delta_v


In [36]:
def calculate_offer_set(capacities_remaining, preference_no_purchase, t, pi, beta=1, dataName=""):
    """
    Implements (14) on p. 777

    :param capacities_remaining:
    :param t:
    :param pi:
    :param beta:
    :return: index of optimal offer set, optimal offer set (the products)
    """
    resources, \
        products, revenues, A, \
        customer_segments, preference_weights, arrival_probabilities, \
        times = get_data_without_variations(dataName)
    lam = sum(arrival_probabilities)

    val_akt = 0
    index_max = 0

    offer_sets_all = get_offer_sets_all(products)
    offer_sets_all = pd.DataFrame(offer_sets_all)

    displacement_costs = displacement_costs_vector(capacities_remaining, preference_no_purchase, t + 2, pi, beta)

    for index, offer_array in offer_sets_all.iterrows():
        val_new = 0
        purchase_rate = purchase_rate_vector(tuple(offer_array), preference_weights,
                                             preference_no_purchase, arrival_probabilities)
        for j in products:
            if offer_array[j] > 0 and all(capacities_remaining - A[:, j] >= 0):
                val_new += purchase_rate[j] * \
                           (revenues[j] - sum(displacement_costs*A[:, j]))
        val_new = lam*val_new

        if val_new > val_akt:
            index_max = copy.copy(index)
            val_akt = copy.deepcopy(val_new)

    return index_max, products[np.array(offer_sets_all.iloc[[index_max]] == 1)[0]] + 1, offer_sets_all


Damit können wir die Ergebnisse aus Bront et al, Table 3 exakt reproduzieren.

<span style="color:red"> Bront et al verwendet die displacement costs von t+2! Warum nicht von t+1? </span>

In [45]:
dfTable3DPD = pd.read_pickle("table3_DPD-offerset_bront-1.pkl")
dfTable3DPD

,rem cap,offer set
0,"[2, 2, 1]","[1, 3, 4, 5]"
1,"[2, 1, 2]","[1, 2, 3, 4, 6]"
2,"[1, 2, 2]","[1, 3, 4, 5]"
3,"[2, 1, 1]","[1, 3, 4]"
4,"[1, 2, 1]","[1, 3, 4, 5]"
5,"[1, 1, 2]","[1, 2, 3, 4, 6]"
6,"[2, 1, 0]","[1, 3]"
7,"[2, 0, 1]","[2, 3, 4]"
8,"[1, 2, 0]","[1, 3, 5]"
9,"[0, 2, 1]","[1, 4, 5]"


In [46]:
dfTable3DPD = pd.read_pickle("table3_DPD-offerset_bront-2.pkl")
dfTable3DPD

,rem cap,offer set
0,"[3, 2, 2]","[1, 3, 4, 5]"
1,"[2, 3, 2]","[1, 3, 4, 5]"
2,"[2, 2, 3]","[1, 2, 3, 4, 5]"
3,"[3, 2, 1]","[1, 3, 4, 5]"
4,"[3, 1, 2]","[1, 2, 3, 4, 6]"
5,"[2, 3, 1]","[1, 3, 4, 5]"
6,"[1, 3, 2]","[1, 3, 4, 5]"
7,"[2, 1, 3]","[1, 2, 3, 4, 6]"
8,"[1, 2, 3]","[1, 3, 4, 5]"
9,"[3, 1, 1]","[1, 3, 4, 6]"


# Talluri and van Rzyin

Hier implementieren wir die Berechnung effizienter Offersets und darauf aufbauend die nested policies.

## efficient_sets

Wir berechnen die efficient sets nach dem Ansatz der marginal revenue ratio. <span style="color:red"> Die Inputvariablen beginnen stets mit dem Nullset. </span> Sei $ES$ die Menge der effizienten Sets. Dann berechnen wir wie folgt:

1. $ES = \{0\}$
2. Berechne $q_{ES} = \max_{S \in ES} Q(S)$
3. Berechne $r_{ES} = \max_{S \in ES} R(S)$
4. Sei $M$ die Menge aller Sets $S'$ mit $Q(S') \geq q_{ES}$ und $R(S') \geq r_{ES}$ und $S' \not\in ES$, dann:
    1. Falls $|M| = 0$: Gib $ES$ zurück. 
    2. Sonst $S^* = \arg\max_{S' \in M} \frac{R(S') - r_{ES}}{Q(S') - q_{ES}}$ und füge $S^*$ zu $ES$ hinzu.

In [25]:
def efficient_sets(data_sets):
    """
    Calculates the efficient sets by marginal revenue ratio as discussed in Talluri and van Ryzin p. 21 upper left

    :param data_sets: A dataset with the quantities and revenues of all sets. Index has the sets.
    :return:
    """
    ES = ['0']
    data_sets = data_sets.sort_values(['q'])
    sets_quantities = data_sets.loc[:, 'q']
    sets_revenues = data_sets.loc[:, 'r']

    while True:
        print(ES)
        q_max = max(sets_quantities[ES])
        r_max = max(sets_revenues[ES])
        tocheck = set(data_sets.index[(sets_quantities >= q_max) & (sets_revenues >= r_max)])
        tocheck = tocheck - set(ES)
        if len(tocheck) == 0:
            return ES
        marg_revenues = pd.DataFrame(data=np.zeros(len(tocheck)), index=tocheck)
        for i in tocheck:
            marg_revenues.loc[i] = (sets_revenues[i] - r_max) / (sets_quantities[i] - q_max)
        ES.append(max(marg_revenues.idxmax()))

# Approximate Policy Iteration

Nun zum Herzstück der Übung. Ich möchte die *Least squares approximate policy iteration* aus Kapitel 3 Koch nachimplementieren. Hier zunächst ein paar Vorworte zur Idee.

## Linear Value Function Approximation

Wir wollen die Wertfunktion approximieren mittels
$$ \hat V_t(c) = \theta_t + \sum_{i\in [m]} \pi_{ti} c_i \quad \forall t, c \geq 0 \quad .$$
Weiter können wir nutzen, dass $\theta_t \geq 0 ~ \forall t$ (no negative value) und dass $\max_{j\in[n]} r_j \geq \pi_{ti} \geq 0 ~ \forall t, h$ (eine Kapazität von einer Ressource kann nicht mehr wert sein als das teuerste Produkt). Außerdem folgt aus $V_{T+1}(c) = 0$, dass $\theta_{T+1} = \pi_{T+1, i} = 0 ~ \forall i$. 

Dies ist also im Wesentliche eine Regression, für welche wir $I$ Testfälle hernehmen.

## Piecewise Linear Value Function Approximation

Da die lineare Version ein Spezialfall der piecewise linear Version ist (nur ein Intervall je Ressource), fokussieren wir uns sofort auf die piecewise linear value function.

Hier kann sich der Wert einer Kapazitätseinheit ändern, je nach aktuellem Level der Kapazität. Dazu wird die Anfangskapazität der Ressource $i$ in $S^i+1$ Intervalle aufgeteilt mit $b_i^s$ als Thresholds, wobei $0 = b_i^0 < ... < b_i^{S^i} = c_i^0$ . Es gilt:

\begin{equation}
f_{is}(c_i) = \begin{cases}
0 & \text{if } c_i \leq b_i \\
c_i - b_i^{s-1} & \text{if } b_i^{s-1} < c_i \leq b_i^s \\
b_i^s - b_i^{s-1} & \text{if } b_i^s < c_i 
\end{cases} \quad \forall s = 1, ..., S^i
\end{equation}

Damit ergibt sich als Schätzung für die Wertfunktion
$$ \hat V_t(c) = \theta_t + \sum_{i\in [m]}\sum_{s \in [S^i]} \pi_{tis} f_{is}(c_i) \quad \forall t, c \geq 0$$
wobei auch hier gilt dass $\theta_t \geq 0 ~ \forall t$ (no negative value) und dass $\max_{j\in[n]} r_j \geq \pi_{tis} \geq 0 ~ \forall t, h, s$ (eine Kapazität von einer Ressource kann nicht mehr wert sein als das teuerste Produkt). Hinzu kommt, dass $\pi_{tis} \geq \pi_{ti, s+1} ~ \forall t,i, s = 1, ..., S^i-1$ Außerdem folgt aus $V_{T+1}(c) = 0$, dass $\theta_{T+1} = \pi_{T+1, is} = 0 ~ \forall i, s$. 

Als finaler bid price für Ressource $i$ wird dann derjenige des jeweils aktuellen Kapazitätsintervalls $c_i$ verwendet: $\pi_{ti}(c_i) = \sum_{s \in [S^i]} \pi_{tis} \cdot \mathbb{1}_{(b_i^{s-1}, b_i^s]}(c_i)$

## Approximate Policy Iteration

Um später die Werte von $\theta_t$ und $\pi_{ti}$ via linearer Regression zu lernen, müssen zunächst Datenpunkte generiert werden. Hierzu werden in jeder Iteration $k$ ($k\in [K]$) zahlreiche, nämlich $P$, sample paths $\omega^p, ~p\in [P]$ erzeugt. 

Betrachten wir eine Iteration. Zu Beginn werden

\begin{equation}
f_{is} = \begin{cases}
0 & \text{if } c_i \leq b_i \\
c_i - b_i^{s-1} & \text{if } b_i^{s-1} < c_i \leq b_i^s \\
b_i^s - b_i^{s-1} & \text{if } b_i^s < c_i 
\end{cases}
\end{equation}

## Implementierung

Bei der Implementierung habe ich mich für eine Implementierung mit möglichst vielen Zwischenergebnissen entschieden. Im Prinzip wird ein großes Datenobjekt gefüllt. 

## Parameter

